In [ ]:
import json
import pyspark.sql.functions as f

In [ ]:
#PARAMETERS
raw_folderpath = ""
raw_filename = ""
mappingJSON = ""
sinkdbTableName = ""
delimiter = ""

In [ ]:
#Storage Config
storageLinkedService = 'LS_DataLake'
storageAccount_ls = mssparkutils.credentials.getPropertiesAll(storageLinkedService)
storage_account = json.loads(storageAccount_ls)['Endpoint'].split('.')[0].replace('https://','')
container = 'raw'

In [ ]:
# Read Data
file_type = raw_filename.split('.')[1]

if file_type == 'json':
    initialDataframe = spark.read.json('abfss://'+container+'@'+ storage_account + '.dfs.core.windows.net/'+ raw_folderpath+'/'+ raw_filename)
elif file_type == 'csv':
    initialDataframe = spark.read.csv('abfss://'+container+'@'+ storage_account + '.dfs.core.windows.net/'+ raw_folderpath+'/'+ raw_filename,sep=delimiter,header=True)

In [ ]:
# Define Mapping Function
def MapData(mappingJSON, container, storage_account, folderpath, filename, table):

    #Step 1: Parse Mapping JSON
    mapping_json = json.loads(mappingJSON)
    columnMappingsDict = list(mapping_json['mappings'])
    columnMappings = [(columnMapping['source'], columnMapping['sink']) for columnMapping in columnMappingsDict]

    #Step 2: Rename Columns
    updated_columns = [f.col(mapping[0]).alias(mapping[1]) for mapping in columnMappings]
    newDataframe=initialDataframe.select([c.cast('string') for c in updated_columns])

    return newDataframe

In [ ]:
#Map DF
mapped_df = MapData(mappingJSON, container, storage_account, raw_folderpath, raw_filename, sinkdbTableName)

In [ ]:
#Output to Staging
mapped_df.write.mode("append").parquet('abfss://'+'staging'+'@'+ storage_account + '.dfs.core.windows.net/EnterpriseTaxModel/'+sinkdbTableName)